In [15]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
    Union,
    List
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.prebuilt import ToolNode
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_ollama import ChatOllama
from langchain_core.messages import ToolMessage,SystemMessage
from langchain_core.runnables import RunnableConfig
from pydantic import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import json

class VerifierBase(BaseModel):
    step: str = Field(..., description="The step as given in the experiment.")
    search_query: str = Field(..., description="The phrase or keywords you would search on the web to verify the step.")
    verification: str = Field(..., description="The expected correct verification result based on technical knowledge and search results.")

class VerifierStructure(BaseModel):
    steps: List[VerifierBase] = Field(..., description="A list of verification steps, each containing step details, search query, and verification result.")

class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]
    generated_exps: dict
    exp_title:str
    reward:Union[int,float]
    verified_structure: list
    tool_results : dict

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wrapper = DuckDuckGoSearchAPIWrapper(max_results=3)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format='list')

tools = [search,wikipedia]

model = ChatOllama(model="qwen2.5:32b",base_url="192.168.23.138:11439")
tool_model = model.bind_tools(tools)
verifier_model = model.with_structured_output(VerifierStructure,method='json_schema')

In [16]:
verifier_system_prompt = '''You are a Verifier AI responsible for checking the accuracy of experimental steps across various scientific and technical domains. Your task is to verify whether each step is correct by using reasonable web searches.

For each step in the experiment, you will:

Identify the key concept(s) involved.
Generate an appropriate search query to verify the correctness of the step.
Validate the step by using the search results to confirm or correct the instruction.
Your output should be structured in JSON format with the following fields:

"step": The step as given in the experiment.
"search_query": The phrase or keywords you would search on the web to verify the step.
"verification": The expected correct verification result based on technical knowledge and search results.
Be precise, factual, and ensure that the verification aligns with established scientific and technical principles'''

In [17]:
def seed_generator(state:AgentState):
    exp_generated = model.invoke(state['messages'])
    return {"generated_exps":{state["exp_title"]:exp_generated.content}}

def verifier_generator(state:AgentState):
    system_prompt = SystemMessage(verifier_system_prompt)
    struct = verifier_model.invoke([system_prompt]+[str(state["generated_exps"])])
    # print(struct.steps)
    
    return {"verified_structure":struct.steps}
tools_by_name = {tool.name: tool for tool in tools}
def tool_decider_node(state:AgentState):
    system_prompt = '''you are given the following 
    1. step: which a step from the experiment
    2. search_query: the query which you have to search 
    you have to make a tool call or tool calls(multiple) which might give the best possible answer try to use all tools which are provided'''
    all_step_tool_results =[]
    for s in state['verified_structure']:
        query = dict(s)
        query.pop('verification')
        tool_results = {"step": query["step"]}
        resp = tool_model.invoke([SystemMessage(system_prompt)]+ [str(query)])
        all_step_tool_results.append(tool_results)
        for tool_call in resp.tool_calls:
            tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
            if tool_call["name"] == 'duckduckgo_results_json':
                
                snippets = []
                for snippet in tool_result:
                    snippets.append(snippet['snippet'])
                tool_results.update({"duckduckgo": snippets})
            elif tool_call["name"] == "wikipedia":
                # print(type(tool_result))
                tool_results.update({"wiki": tool_result})
            # print(tool_result)
        
    return{"tool_results":all_step_tool_results}

In [18]:
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)
workflow.add_node("generator",seed_generator)
workflow.add_node("verifier",verifier_generator)
workflow.add_node("toolcaller",tool_decider_node)
workflow.set_entry_point("generator")
workflow.set_finish_point("toolcaller")
workflow.add_edge("generator","verifier")
workflow.add_edge("verifier","toolcaller")
graph = workflow.compile()

In [19]:
exp = "To set up and test a unity gain amplifier using an IC 741 operational amplifier."

query = f"what should be the required materials and experiment procedure of the following lab Experiment ({exp})?"
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", query)], "exp_title":exp,"verified_structure":[]}
final = graph.invoke(inputs)
# print_stream(graph.stream(inputs, stream_mode="values"))

In [27]:
final['tool_results']

[{'step': 'Insert the IC 741 into the breadboard ensuring that pin 1 (the offset null) is oriented correctly.',
  'duckduckgo': ["Pinout Of IC 741 Op Amp (Pin Diagram) An IC 741 op-amp package consists of 8 pins. The below IC 741 Pin Diagram illustrates the pin configurations and internal block diagram of IC 741 in 8 pin DIP and TO5-8 metal can package. 741 IC Op Amplifier Circuit Functions. Now let's take a look at the functions of different pins of 741 IC op-amp circuits:",
   'The steps to build the non-inverting op amp circuit are to first place IC 741 on the breadboard. Connect pin 3 directly to the signal source. Connect pin 3 to the ground through resistor R1. Connect pin 6 to pin 3 via resistor Rf. Connect pin 4 to ground and pin 7 to +15V. Apply a test signal and observe the amplified output on an oscilloscope.',
   'An in-depth overview of 741 Op Amp IC, its characteristics, pin configuration, and applications. Understand its parameters and how it compares to alternatives lik

In [23]:
len([dict(i) for i in final['verified_structure']])

7

In [25]:
md_content = final['generated_exps'][exp]

In [26]:
from IPython.display import display, Markdown
display(Markdown(md_content))

Setting up and testing a unity gain amplifier using an IC 741 operational amplifier is a common laboratory exercise in electronics. Below are the required materials and the detailed experiment procedure.

### Required Materials:
1. **IC 741 Operational Amplifier** - This is the main component that will perform the amplification.
2. **Breadboard** - Used to easily connect and test circuits without soldering.
3. **DC Power Supply (±15V)** - The IC 741 requires a dual power supply for its operation; ±15V is often recommended.
4. **Resistors**: Two resistors of the same value, typically 10kΩ each are used in this setup to ensure unity gain.
5. **Potentiometer (optional)** - Can be used if you want to adjust the input signal for testing purposes.
6. **Signal Generator** - To provide an AC input signal for testing the amplifier's performance.
7. **Oscilloscope** - Essential for measuring and observing the input and output signals.
8. **Connecting Wires** - To connect different components on the breadboard.

### Experiment Procedure:

1. **Preparation:**
   - Assemble all necessary materials before starting your experiment.
   - Ensure that you understand how to use an oscilloscope, signal generator, and other electronic equipment in your lab.

2. **Setting Up the Circuit:**
   - Insert the IC 741 into the breadboard ensuring that pin 1 (the offset null) is oriented correctly. This will often be marked with a dot or half-moon shape on the component itself.
   - Connect pin 4 of the IC to ground (0V).
   - Pin 2 should be connected to your input signal through one resistor, and also connect this same resistor from pin 3 (output) back to pin 2. This forms a unity gain buffer configuration where the output is equal in magnitude but in phase with the input.
   - Connect pin 7 (+Vcc) to +15V of the DC power supply and pin 8 (-Vcc) to -15V.

3. **Connecting Input Signal:**
   - Use your signal generator to produce an AC voltage output, which you will feed into pin 2 (the non-inverting input).
   - Adjust the frequency and amplitude as needed for testing purposes.
   
4. **Testing the Amplifier:**
   - Connect the oscilloscope probes:
     - One probe on the input signal line just before it reaches IC741's pin 2 to observe the original input signal.
     - Another probe on pin 3 of the IC, which is your output signal.
   - Observe both signals on the oscilloscope. The goal here is for the amplitude and frequency of the two waveforms to be identical (within a small margin due to real-world component tolerances), indicating that you've achieved unity gain.

5. **Data Recording:**
   - Record the input voltage and output voltage values observed using the oscilloscope.
   - Note any discrepancies or anomalies in your measurements for further analysis.

6. **Analysis:**
   - Compare the input and output signals. Ideally, with a properly constructed circuit, the ratio of the peak-to-peak amplitude of the output signal to that of the input signal should be close to 1 (or unity gain).
   - Discuss any variations from expected results considering factors such as component tolerances or environmental noise.

7. **Conclusion:**
   - Summarize your findings and discuss the success rate of achieving a unity gain amplifier.
   - Reflect on possible improvements for future setups if there were any issues encountered during testing.

This setup provides you with a foundational understanding of operational amplifiers and how they can be used in simple configurations like buffers, which are essential in many practical applications.